In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import time


In [2]:
# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [3]:
# Flatten + normalize
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

Xtr = x_train.reshape(len(x_train), -1)
Xval = x_val.reshape(len(x_val), -1)
Xte = x_test.reshape(len(x_test), -1)
y_train = y_train.reshape(-1)
y_val = y_val.reshape(-1)
y_test = y_test.reshape(-1)
print(y_val.shape)

(10000,)


In [4]:
# Standardize

scaler = StandardScaler()
Xtr_s = scaler.fit_transform(Xtr)
Xval_s = scaler.transform(Xval)
Xte_s = scaler.transform(Xte)

In [5]:
# PCA
pca = PCA(n_components=0.90, random_state=0)
Xtr_pca = pca.fit_transform(Xtr_s)
Xval_pca = pca.transform(Xval_s)
Xte_pca = pca.transform(Xte_s)

print("PCA dimensions:", Xtr_pca.shape)

PCA dimensions: (40000, 102)


In [6]:
# Subset γιατι εσκαγε η RAM

Xtr_sub, _, ytr_sub, _ = train_test_split(Xtr_pca, y_train, train_size=10000, stratify=y_train, random_state=0)

In [7]:
# RBF κεντρα με KMeans
M = 2000   # πληθος RBF νευρωνων - κεντρων
t0 = time.perf_counter()
kmeans = KMeans(n_clusters=M, random_state=0)
kmeans.fit(Xtr_pca)
cen_time = time.perf_counter()-t0
print(f"Kmeans time: {cen_time:.2f} sec")
C = kmeans.cluster_centers_

Kmeans time: 67.22 sec


In [24]:
# RBF Layer
d2_tr  = ((Xtr_sub[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_val = ((Xval_pca[:, None, :] - C[None, :, :])**2).sum(axis=2)

# For various sigma
for sigma in [20, 40, 60, 80, 100]:
    t0 = time.perf_counter()
    denom = 2.0 * sigma * sigma
    Phi_tr  = np.exp(-d2_tr / denom)
    Phi_val = np.exp(-d2_val / denom)

    clf = LogisticRegression(max_iter=3000, n_jobs=-1)
    clf.fit(Phi_tr, ytr_sub)

    val_acc = accuracy_score(y_val, clf.predict(Phi_val))

    t1 = time.perf_counter() - t0
    print("sigma:", sigma, "val_acc:", val_acc, "time: " f"{t1:.2f} sec" )


sigma: 20 val_acc: 0.4132 time: 24.54 sec
sigma: 40 val_acc: 0.4379 time: 101.71 sec
sigma: 60 val_acc: 0.4203 time: 178.38 sec
sigma: 80 val_acc: 0.4038 time: 178.63 sec
sigma: 100 val_acc: 0.3883 time: 274.98 sec


In [27]:
# Narrow search

for sigma in [30, 35, 40, 45, 50]:
    t0 = time.perf_counter()
    denom = 2.0 * sigma * sigma
    Phi_tr  = np.exp(-d2_tr / denom)
    Phi_val = np.exp(-d2_val / denom)

    clf = LogisticRegression(max_iter=3000, n_jobs=-1)
    clf.fit(Phi_tr, ytr_sub)

    val_acc = accuracy_score(y_val, clf.predict(Phi_val))

    t1 = time.perf_counter() - t0
    print("sigma:", sigma, "val_acc:", val_acc, "time: " f"{t1:.2f} sec")

sigma: 30 val_acc: 0.4401 time: 64.34 sec
sigma: 35 val_acc: 0.441 time: 84.85 sec
sigma: 40 val_acc: 0.4379 time: 98.64 sec
sigma: 45 val_acc: 0.4351 time: 129.90 sec
sigma: 50 val_acc: 0.4311 time: 152.32 sec


In [8]:
# Best sigma

sigma = 40
denom = 2.0 * sigma * sigma

t0 = time.perf_counter()

d2_tr  = ((Xtr_sub[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_val = ((Xval_pca[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_te  = ((Xte_pca[:, None, :] - C[None, :, :])**2).sum(axis=2)

Phi_tr  = np.exp(-d2_tr / denom)
Phi_val = np.exp(-d2_val / denom)
Phi_te  = np.exp(-d2_te / denom)

t_rbf = time.perf_counter() - t0
print(f"RBF feature time: {t_rbf:.2f} sec")


t0 = time.perf_counter()

clf = LogisticRegression(max_iter=3000, n_jobs=-1)
clf.fit(Phi_tr, ytr_sub)


t_train = time.perf_counter() - t0
print(f"Classifier training time: {t_train:.2f} sec")

t0 = time.perf_counter()

y_pred_val = clf.predict(Phi_val)
y_pred_test = clf.predict(Phi_te)

t_pred = time.perf_counter() - t0
print(f"Prediction time: {t_pred:.2f} sec")

val_acc = accuracy_score(y_val, y_pred_val)
test_acc = accuracy_score(y_test,  y_pred_test)
print("Validation acc:", val_acc)
print("Test acc:", test_acc)
print(f"Total RBF pipeline time: {t_rbf + t_train + t_pred:.2f}")

RBF feature time: 22.92 sec
Classifier training time: 102.87 sec
Prediction time: 0.28 sec
Validation acc: 0.4379
Test acc: 0.4407
Total RBF pipeline time: 126.07


# **My K-means**

In [7]:
# Center initialization Function

def center_initialization(X, K, random_state=0):

  np.random.seed(random_state)
  N, F = X.shape
  idx = np.random.choice(N, K, replace=False)
  centers = X[idx, :]

  return centers

In [8]:
# Example

centers = center_initialization(Xtr_pca, K=1000)
print(centers.shape)

(1000, 102)


In [9]:
# Clustering-Labeling Function

def clustering(X, centers):
    N, F = X.shape
    K = centers.shape[0]

    distances = np.zeros((N, K))

    for i in range(N):          # για καθε δειγμα
        for j in range(K):      # για καθε κεντρο
            distances[i, j] = np.sum((X[i]-centers[j]) ** 2)    # distances: (N,K)

    labels = np.argmin(distances, axis=1)   # Σκαναρε το distances matrix οριζοντια, παρε το index της στηλης (cluster)
                                            # που δινει ελαχιστη αποσταση του δειγματος xi απο το κεντρο cj

    return labels    # (N,): διανυσμα Ν γραμμων, το στοιχειο στη γραμμη i ειναι το cluster στο οποιο ανηκει το δειγμα x[i]

In [10]:
# Example

labels = clustering(Xtr_sub, centers)
print(labels)

[389  63 691 ... 213 147 319]


In [11]:
# Function that finds new centers (and updates them) as the mean of all training samples in one cluster

def update_centers(X, labels, K, random_state=0):
    np.random.seed(random_state)
    N, F = X.shape
    new_centers = np.zeros((K, F), dtype=X.dtype)

    for j in range(K):    # Για το κεντρο j
        # μαζεψε ολα τα σημεια του cluster j
        points = []   # λιστα, αδειαζει σε καθε επαναληψη του loop, δηλαδη για καθε cluster
        for i in range(N):   # Για τα δειγματα i
            if labels[i] == j:  # που τα labels τους ειναι ισα με j, δηλαδη ανηκουν στο cluster j
                points.append(X[i])     # Γεμισε τη λιστα με τα δειγματα x[i], αρα points: (Nj, F)
                                        # οπου Nj δειγματα που ανηκουν στο cluster j και F τα features

        # ενημερωση κεντρου
        if len(points) == 0:    # Αν το cluster j ειναι αδειο
            new_centers[j] = X[np.random.randint(N)]    # θεσε το κεντρο ως ενα τυχαιο δειγμα
        else:
            new_centers[j] = np.mean(points, axis=0)  # το νεο κεντρο θα ειναι ο μεσος ορος των δειγματων στο cluster j

    return new_centers    # (K,F)

In [12]:
# Completed kmeans that repeats the two optimization methods above until the centers don't change much (<tol)

def mykmeans(X, K, tol, max_iters, random_state=0):

    centers = center_initialization(X, K, random_state=random_state)

    for it in range(max_iters):

        labels = clustering(X, centers)

        new_centers = update_centers(X, labels, K, random_state=random_state)

        shift = np.sqrt(np.sum((new_centers - centers) ** 2))   # Μετακινηση κεντρων

        centers = new_centers

        if shift < tol:
            break

    return centers

In [13]:
K = 200
t0 = time.perf_counter()
centers = mykmeans(Xtr_sub, K, tol=1e-3, max_iters=20, random_state=0)
t_centers = time.perf_counter() - t0
print(f"My Kmeans time: {t_centers:.2f} sec")

My Kmeans time: 280.70 sec


In [14]:
print("Xtr_sub:", Xtr_sub.shape)
print("Xval_pca:", Xval_pca.shape)
print("Xte_pca:", Xte_pca.shape)
print("ytr_sub:", ytr_sub.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

Xtr_sub: (10000, 102)
Xval_pca: (10000, 102)
Xte_pca: (10000, 102)
ytr_sub: (10000,)
y_val: (10000,)
y_test: (10000,)


In [15]:
sigma = 40
denom = 2.0 * sigma * sigma

C = centers

t0 = time.perf_counter()
d2_tr = ((Xtr_sub[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_val = ((Xval_pca[:, None, :]  - C[None, :, :])**2).sum(axis=2)
d2_test = ((Xte_pca[:, None, :]  - C[None, :, :])**2).sum(axis=2)

Phi_tr  = np.exp(-d2_tr / denom)
Phi_val  = np.exp(-d2_val / denom)
Phi_test = np.exp(-d2_test / denom)

t_rbf = time.perf_counter() - t0
print(f"RBF feature time: {t_rbf:.2f} sec")

t0 = time.perf_counter()
clf = LogisticRegression(max_iter=3000, n_jobs=-1)
clf.fit(Phi_tr, ytr_sub)
t_train = time.perf_counter() - t0
print(f"Classifier training time: {t_train:.2f} sec")

t0 = time.perf_counter()

y_pred_val = clf.predict(Phi_val)
y_pred_test = clf.predict(Phi_test)

t_pred = time.perf_counter() - t0
print(f"Prediction time: {t_pred:.2f} sec")

val_acc = accuracy_score(y_val, y_pred_val)
test_acc = accuracy_score(y_test,  y_pred_test)
print("Validation acc:", val_acc)
print("Test acc:", test_acc)
print(f"Total RBF pipeline time: {t_rbf + t_train + t_pred:.2f}")

RBF feature time: 2.26 sec
Classifier training time: 8.04 sec
Prediction time: 0.04 sec
Validation acc: 0.3833
Test acc: 0.3839
Total RBF pipeline time: 10.34


# **RBF without Kmeans**
The centers are now random training samples

In [ ]:
def sample_centers(X, M, random_state=0):
    rng = np.random.default_rng(random_state)
    idx = rng.choice(len(X), size=M, replace=False)
    return X[idx]

In [ ]:
M = 2000
t0 = time.perf_counter()
C = sample_centers(Xtr_sub, M, random_state=0)
t_centers = time.perf_counter() - t0
print(f"Centers time: {t_centers:.2f} sec")

Centers time: 0.00 sec


In [ ]:
sigma = 40
denom = 2.0 * sigma * sigma

t0 = time.perf_counter()
d2_tr   = ((Xtr_sub[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_val  = ((Xval_pca[:, None, :] - C[None, :, :])**2).sum(axis=2)
d2_test = ((Xte_pca[:, None, :]  - C[None, :, :])**2).sum(axis=2)

Phi_tr   = np.exp(-d2_tr / denom)
Phi_val  = np.exp(-d2_val / denom)
Phi_test = np.exp(-d2_test / denom)

t_rbf = time.perf_counter() - t0
print(f"RBF feature time: {t_rbf:.2f} sec")

t0 = time.perf_counter()
clf = LogisticRegression(max_iter=3000, n_jobs=-1)
clf.fit(Phi_tr, ytr_sub)

t_train = time.perf_counter() - t0
print(f"Classifier training time: {t_train:.2f} sec")

t0 = time.perf_counter()

y_pred_val = clf.predict(Phi_val)
y_pred_test = clf.predict(Phi_test)

t_pred = time.perf_counter() - t0
print(f"Prediction time: {t_pred:.2f} sec")

val_acc = accuracy_score(y_val, y_pred_val)
test_acc = accuracy_score(y_test,  y_pred_test)
print("Validation acc:", val_acc)
print("Test acc:", test_acc)
print(f"Total RBF pipeline time: {t_rbf + t_train + t_pred:.2f}")

RBF feature time: 15.91 sec
Classifier training time: 62.88 sec
Prediction time: 0.20 sec
Validation acc: 0.448
Test acc: 0.4458
Total RBF pipeline time: 78.98


# **CNN as Feature Extractor + RBF**

In [ ]:
cnn = tf.keras.models.load_model("final_model6.h5")
cnn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,118,764 (4.27 MB)

 Trainable params: 1,117,866 (4.26 MB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 2 (12.00 B)

In [ ]:
(x_train2, y_train2), (x_test2, y_test2) = cifar10.load_data()

In [ ]:
# Flatten + normalize
x_train2 = x_train2.astype("float32") / 255.0
x_test2 = x_test2.astype("float32") / 255.0

X_train2, X_val2, Y_train2, Y_val2 = train_test_split(x_train2, y_train2, test_size=0.2, random_state=42, stratify = y_train2)

y_train_re = Y_train2.reshape(-1)
y_val_re = Y_val2.reshape(-1)
y_test_re = y_test2.reshape(-1)
print(y_val_re.shape)

(10000,)


In [ ]:
feature_extractor = tf.keras.Model(inputs=cnn.inputs, outputs=cnn.get_layer("activation_4").output)

In [ ]:
feature_extractor.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,048,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 256)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,116,192 (4.26 MB)

 Trainable params: 1,115,296 (4.25 MB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
Xtr_feat  = feature_extractor.predict(X_train2, batch_size=256, verbose=1)
Xval_feat = feature_extractor.predict(X_val2, batch_size=256, verbose=1)
Xte_feat  = feature_extractor.predict(x_test2, batch_size=256, verbose=1)

print(Xtr_feat.shape, Xval_feat.shape, Xte_feat.shape)

157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
(40000, 256) (10000, 256) (10000, 256)


In [ ]:
# Scaling the cnn outputs before RBF

scaler = StandardScaler()
Xtr_feat_s  = scaler.fit_transform(Xtr_feat)
Xval_feat_s = scaler.transform(Xval_feat)
Xte_feat_s  = scaler.transform(Xte_feat)

In [ ]:
M = 300
t0 = time.perf_counter()
kmeans2 = KMeans(n_clusters=M, random_state=0)
kmeans2.fit(Xtr_feat_s)
C2 = kmeans2.cluster_centers_
t_centers = time.perf_counter() - t0
print(f"Centers time: {t_centers:.2f} sec")

Centers time: 28.90 sec


In [ ]:
Xtr_feat_s = Xtr_feat.astype(np.float32)
Xval_feat_s = Xval_feat.astype(np.float32)
Xte_feat_s = Xte_feat.astype(np.float32)
C = C2.astype(np.float32)

In [ ]:
# Subset

Xtr_sub2, _, ytr_sub2, _ = train_test_split(Xtr_feat_s, y_train, train_size=12000, stratify=y_train, random_state=0)

In [ ]:
t0 = time.perf_counter()
d2_tr2   = ((Xtr_sub2[:, None, :] - C[None, :, :])**2).sum(axis=2).astype(np.float32)
d2_val2  = ((Xval_feat_s[:, None, :] - C[None, :, :])**2).sum(axis=2).astype(np.float32)
d2_test2 = ((Xte_feat_s[:, None, :]  - C[None, :, :])**2).sum(axis=2).astype(np.float32)

sigma = 30
denom = np.float32(2.0 * sigma * sigma)
Phi_tr2  = np.exp(-d2_tr2 / denom)
Phi_val2 = np.exp(-d2_val2 / denom)
Phi_te2  = np.exp(-d2_test2 / denom)

t_rbf = time.perf_counter() - t0
print(f"RBF feature time: {t_rbf:.2f} sec")

t0 = time.perf_counter()
clf = LogisticRegression(max_iter=3000, n_jobs=-1)
clf.fit(Phi_tr2, ytr_sub2)

t_train = time.perf_counter() - t0
print(f"Classifier training time: {t_train:.2f} sec")

t0 = time.perf_counter()
y_tr_pred = clf.predict(Phi_tr2)
y_val_pred = clf.predict(Phi_val2)
y_test_pred = clf.predict(Phi_te2)

t_pred = time.perf_counter() - t0
print(f"Prediction time: {t_pred:.2f} sec")

tr_acc = accuracy_score(ytr_sub2, y_tr_pred)
val_acc = accuracy_score(y_val_re, y_val_pred)
test_acc = accuracy_score(y_test_re, y_test_pred)
print("Training accuracy:", tr_acc)
print("Validation accuracy:", val_acc)
print("Test accuracy:",  test_acc)

RBF feature time: 8.63 sec
Classifier training time: 8.81 sec
Prediction time: 0.05 sec
Training accuracy: 0.8676666666666667
Validation accuracy: 0.831
Test accuracy: 0.8032
